In [ ]:
%pip install xhtml2pdf

In [26]:
from IPython.display import display, HTML
from xhtml2pdf import pisa 
from dotenv import load_dotenv
from sqlalchemy import create_engine
from datetime import datetime, timedelta
from os import getenv
import pandas as pd

In [15]:
load_dotenv()

True

In [16]:
db_host = getenv('DB_HOST')
db_port = getenv('DB_PORT')
db_user = getenv('DB_USER')
db_password = getenv('DB_PASSWORD')
db_name = getenv('DB_NAME')
group_user = getenv('GROUP_USER')
group_user_pass = getenv('GROUP_USER_PASS')

In [17]:
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [20]:
con = engine.connect()

In [27]:
def get_current_date() -> str:
   """
   Returns the current date (note :currently set as yesterdays date)
   """
   current_date = str(datetime.now().date() - timedelta(days = 1))
   return current_date
current_date = get_current_date()
current_date

'2022-10-07'

In [46]:
query = """
SELECT * FROM yusra_stories_production.rides
"""
pd.read_sql_query(query, con)

,ride_id,user_id,start_time,end_time,total_duration,max_heart_rate_bpm,min_heart_rate_bpm,avg_heart_rate_bpm,avg_resistance,avg_rpm,total_power_kilojoules
0,1,4651,2022-10-07 15:08:31,2022-10-07 15:17:13,0:08:40,157,71,107,41,47,18.01
1,2,4651,2022-10-07 15:17:14,2022-10-07 15:25:55,0:08:40,151,85,105,41,48,12.75
2,3,4651,2022-10-07 15:25:57,2022-10-07 15:34:38,0:08:40,155,80,121,41,50,13.93
3,5,4653,2022-10-07 15:43:22,2022-10-07 15:49:17,0:05:54,168,81,133,44,44,14.56
4,4,4653,2022-10-07 15:34:40,2022-10-07 15:43:21,0:08:40,158,80,111,40,41,7.29
5,6,4653,2022-10-07 15:49:19,2022-10-07 15:58:00,0:08:40,157,67,114,39,41,8.53
6,127,4674,2022-10-08 09:06:31,2022-10-08 09:15:12,0:08:40,170,55,107,38,50,13.72


### Quering database

#### Number of rides completed in the past day

In [35]:
query = f"""
WITH rides AS (  
    SELECT *, CAST(start_time AS DATE) AS start_date
    FROM yusra_stories_production.rides
    )
SELECT COUNT(*) AS number_of_rides 
FROM rides 
WHERE start_date = '{current_date}';
"""
number_of_rides = pd.read_sql_query(query, con)
number_of_rides

,number_of_rides
0,6


#### Gender split of riders of the past day

In [49]:
query = f"""
WITH rides AS (  
    SELECT *, CAST(start_time AS DATE) AS start_date
    FROM yusra_stories_production.rides
    ),
riders AS (
SELECT DISTINCT (user_id), name, gender, age
FROM yusra_stories_production.users
JOIN rides
USING (user_id)
WHERE start_date = '{current_date}'
)
SELECT gender, COUNT(*) AS number_of_riders
FROM riders
GROUP BY gender;
"""
riders_gender_split = pd.read_sql_query(query, con)
riders_gender_split

,gender,number_of_riders
0,female,1
1,male,1


#### Ages of the riders of the past day

In [48]:
query = f"""
WITH rides AS (  
    SELECT *, CAST(start_time AS DATE) AS start_date
    FROM yusra_stories_production.rides
    )
SELECT DISTINCT (user_id), name, gender, age
FROM yusra_stories_production.users
JOIN rides
USING (user_id)
WHERE start_date = '{current_date}'
"""
ages_of_riders = pd.read_sql_query(query, con)
ages_of_riders

,user_id,name,gender,age
0,4651,Mohammad Phillips-Simpson,male,38.0
1,4653,Abbie Jones,female,23.0


#### Average power and heart rate of riders of past day

In [51]:
query = f"""
WITH rides AS (  
    SELECT *, CAST(start_time AS DATE) AS start_date
    FROM yusra_stories_production.rides
    )
SELECT user_id, AVG(avg_heart_rate_bpm) AS average_heart_rate, AVG(total_power_kilojoules) AS average_power
FROM yusra_stories_production.users
JOIN rides
USING (user_id)
WHERE start_date = '{current_date}'
GROUP BY user_id
"""
riders_average_power_and_heart_rate = pd.read_sql_query(query, con)
riders_average_power_and_heart_rate

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "yusra_stories_production.rides" does not exist
LINE 4:     FROM yusra_stories_production.rides
                 ^

[SQL: 
WITH rides AS (  
    SELECT *, CAST(start_time AS DATE) AS start_date
    FROM yusra_stories_production.rides
    )
SELECT user_id, AVG(avg_heart_rate_bpm) AS average_heart_rate, AVG(total_power_kilojoules) AS average_power
FROM yusra_stories_production.users
JOIN rides
USING (user_id)
WHERE start_date = '2022-10-07'
GROUP BY user_id
]
(Background on this error at: https://sqlalche.me/e/14/f405)

### Create the HTML Template

In [32]:
def report_block_template( graph_relative_path, caption=''):

    graph_block = (''
            '<a href="{graph_relative_path}" target="_blank">' # Open the interactive graph when you click on the image
                '<img style="height: 400px;" img src="{graph_relative_path}">'
            '</a>')

    report_block = ('' +
        graph_block +
        '{caption}' + 
        '<br>'      + 
        '</a>' +
        '<br>' +
        '<hr>')                       
    report_layout = (
       '<h2>Deloton Exercise Bikes Daily Report</h2>'
       + '<hr>'
       + report_block
    )
    return report_layout.format(graph_relative_path=graph_relative_path, caption=caption)


graph_relative_path = 'age_distr.png'

report = report_block_template(graph_relative_path, caption='Dis a graph')


In [33]:
display(HTML(report))

### Convert the HTML to PDF

In [28]:

def convert_html_to_pdf(source_html, output_filename):

    result_file = open(output_filename, "w+b")

    pisa_status = pisa.CreatePDF(
            source_html,           
            dest=result_file)           

    result_file.close()           

    return pisa_status.err

In [29]:
convert_html_to_pdf(report, 'report.pdf')

0

In [30]:
! open report.pdf